In [1]:
from math import ceil
from spatial_scan import scan, deploy
from vae import VAE, encoder, decoder, tf, keras,l
import numpy as np
import matplotlib.pyplot as plt


2023-05-22 11:03:26.622223: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 11:03:28.234940: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-22 11:03:28.235063: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-22 11:03:28.235078: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
batch_len = 1000
lmax=0.1

with open("../mc-sampling/good-runs/test_30_5_0.05_0.3_0.1_10000_1000_1000_x.txt") as f:
    data = [[float(i.strip()) for i in s.split(" ") if i!=""] for s in f.read().split("\n") if s!=""]
    xyz=[[[data[h][i+3*j]for i in range(3)]for j in range(len(data[h])//3)]for h in range(len(data))]
    del data

# print(len(data), [len(data[i]) for i in range(len(data))])
model_checkpoint_callback = keras.callbacks.ModelCheckpoint("./checkpoints/", save_best_only=True)

def change_l(e,log):
    global l, lmax
    print("l: ", l)
    l=(1-1/(1+e))*lmax

regularization_callback = keras.callbacks.LambdaCallback(on_epoch_begin=change_l)
    

In [3]:
if False:#len(argv)>2 and argv[2]=="--resume":
    encoder=keras.models.load_model("./model-info/enc")
    decoder=keras.models.load_model("./model-info/dec")

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

In [ ]:
for i in range(1):#(ceil(len(xyz)/batch_len)):
    bdata = [] # will hold bucketed data version
    for j in range(batch_len*i, batch_len*(i+1)):
        if j>len(xyz):
            break
        #print(xyz[j])
        bdata.append([scan(xyz[j],5,5,5)])
    dataset = tf.data.Dataset.from_tensor_slices((bdata,))
    print("i: ",i)
    vae.fit(dataset, epochs=2, batch_size=128, callbacks=[model_checkpoint_callback, regularization_callback], verbose=1)

0
0
Epoch 1/2
 286/1000 [=======>......................] - ETA: 2:02 - loss: 0.0300 - reconstruction_loss: 0.0300 - kl_loss: 1.1232

In [ ]:
try:
    vae.load_weights("./checkpoints/")
except Exception as e:
    print(e)
finally:
    keras.models.save_model(vae.encoder, "./model-info/enc/")
    keras.models.save_model(vae.decoder, "./model-info/dec/")
    

In [ ]:
vae.history.history

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(ncols=3)
ax[0].plot(vae.history.history["loss"])
ax[1].plot(vae.history.history["reconstruction_loss"])
ax[2].plot(vae.history.history["kl_loss"])

In [ ]:
test_set=[]
for j in range(batch_len, batch_len*2):
    if j>len(xyz):
        break
    test_set.append([scan(xyz[j],5,5,5)])

dataset = tf.data.Dataset.from_tensor_slices((test_set,))

pred = vae.encoder.predict(dataset)

In [ ]:
fig,ax=plt.subplots()
ax.scatter(x=[p[0] for p in pred[2]], y=[p[1] for p in pred[2]])
#print(len([p[0] for p in pred[2]]), len([p[1] for p in pred[2]]))

In [ ]:
rdata=np.random.normal(0,1, size=(30,2))
pred=decoder.predict(rdata)
x,y,z=deploy(pred)

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(x,y,z)